In [131]:
import numpy as np
import pandas as pd
import sqlite3
import json
from datetime import datetime
from collections import namedtuple

pd.set_option('display.max_columns', None)

In [132]:
DB_PATH = '/Users/ezras/projects/personal/finance_ml/data/quarterly_financial_data.db'
INFO_CSV_PATH = '/Users/ezras/projects/personal/finance_ml/data/stock_general_info.csv'
TABLE_NAME = 'yahoo_financial_data'
STOCKPUP_TABLE_NAME = 'stockpup_data'


DELTA_PREFIX = 'Delta_'
VS_MKT_IDX = '_vs_'
AVG_REC_SCORE_PREFIX = 'AvgRecScore_'

MISSING_SECTOR = 'MissingSector'
MISSING_INDUSTRY = 'MissingIndustry'

MONTH_TO_QUARTER = {
    1: 4,
    2: 1,
    3: 1,
    4: 1,
    5: 2,
    6: 2,
    7: 2,
    8: 3,
    9: 3,
    10: 3,
    11: 4,
    12: 4
}
    

class StockPupColumns:
    """
    Our dataset comes from over 20 years of 10-Q and 10-K filings made by public companies
     with the U.S. Securities and Exchange Commission. We extract data from both text and
     XBRL filings, fix reporting mistakes, and normalize the data into quarterly time series
     of final restated values.
    """
    # Date Quarter Ends
    QUARTER_END = 'QuarterEnd'
    # The total number of common shares outstanding at the end of a given quarter, including all
    # classes of common stock.
    SHARES = 'Shares'
    # The number of shares the company had at the end of a given quarter, adjusted for splits to
    # be comparable to today's shares.
    SHARES_SPLIT_ADJUSTED = 'SharesSplitAdjusted'
    # If an investor started with 1 share of stock at the end of a given quarter, the split factor
    # for that quarter indicates how many shares the investor would own today as a result of
    # subsequent stock splits.
    SPLIT_FACTOR = 'SplitFactor'
    # Total assets at the end of a quarter.
    ASSETS = 'Assets'
    # Current assets at the end of a quarter.
    CURRENT_ASSETS = 'CurrentAssets'
    # Total liabilities at the end of a quarter.
    LIABILITIES = 'Liabilities'
    # Current liabilities at the end of a quarter.
    CURRENT_LIABILITIES = 'CurrentLiabilities'
    # Total shareholders' equity at the end of a quarter, including both common and preferred
    # stockholders.
    SHAREHOLDER_EQUITY = 'ShareholdersEquity'
    # Non-controlling or minority interest, if any, excluded from Shareholders equity.
    NON_CONTROLLING_INTEREST = 'NonControllingInterest'
    # Preferred equity, if any, included in Shareholders equity.
    PREFERRED_EQUITY = 'PreferredEquity'
    # Total Goodwill and all other Intangible assets, if any.
    GOODWILL_AND_INTANGIBLES = 'GoodwillIntangibles'
    # All long-term debt including capital lease obligations.
    LONG_TERM_DEBT = 'LongTermDebt'
    # Total revenue for a given quarter.
    REVENUE = 'Revenue'
    # Earnings or Net Income for a given quarter.
    EARNINGS = 'Earnings'
    # Earnings available for common stockholders - Net income minus earnings that must be
    # distributed to preferred shareholders. May be omitted when not reported by the company.
    EARNINGS_AVAILABLE_FOR_COMMON_STOCKHOLDERS = 'EarningsAvailableForCommonStockholders'
    # Basic earnings per share for a given quarter.
    EPS_BASIC = 'EPS_basic'
    # Diluted earnings per share.
    EPS_DILUTED = 'EPS_diluted'
    # Common stock dividends paid during a quarter per share, including all regular and special
    # dividends and distributions to common shareholders.
    DIVIDEND_PER_SHARE = 'DividendPerShare'
    # Cash produced by operating activities during a given quarter, including Continuing and
    # Discontinued operations.
    CASH_FROM_OPERATING_ACTIVITES = 'CashFromOperatingActivities'
    # Cash produced by investing activities during a given quarter, including Continuing and
    # Discontinued operations.
    CASH_FROM_INVESTING_ACTIVITIES = 'CashFromInvestingActivities'
    # Cash produced by financing activities during a given quarter, including Continuing and
    # Discontinued operations.
    CASH_FROM_FINANCING_ACTIVITES = 'CashFromFinancingActivities'
    # Change in cash and cash equivalents during a given quarter, including Effect of Exchange
    # Rate Movements and Other Cash Change Adjustments, if any.
    CASH_CHANGE_DURING_PERIOD = 'CashChangeDuringPeriod'
    # Cash and cash equivalents at the end of a quarter, including Continuing and
    # Discontinued operations.
    CASH_AT_END_OF_PERIOD = 'CashAtEndOfPeriod'
    # Capital Expenditures are the cash outflows for long-term productive assets, net of cash
    # from disposals of capital assets.
    CAPITAL_EXPENDITURES = 'CapitalExpenditures'
    # The medium price per share of the company common stock during a given quarter. The prices
    # are as reported, and are not adjusted for subsequent dividends.
    PRICE = 'Price'  # Average price during quarter
    # The highest price per share of the company common stock during a given quarter.
    PRICE_HIGH = 'PriceHigh'
    # The lowest price of the company common stock during a quarter.
    PRICE_LOW = 'PriceLow'
    # Return on equity is the ratio of Earnings (available to common stockholders)
    # TTM (over the Trailing Twelve Months) to TTM average common shareholders' equity.
    ROE = 'ROE'
    # Return on assets is the ratio of total Earnings TTM to TTM average Assets.
    ROA = 'ROA'
    # Common stockholders' equity per share, also known as BVPS.
    BOOK_VALUE_OF_EQUITY_PER_SHARE = 'BookValueOfEquityPerShare'
    # The ratio of Price to Book value of equity per share as of the previous quarter.
    P_B_RATIO = 'P_B_ratio'
    # The ratio of Price to EPS diluted TTM as of the previous quarter.
    P_E_RATIO = 'P_E_ratio'
    # The aggregate amount of dividends paid per split-adjusted share of common stock from the
    # first available reporting quarter until a given quarter.
    CUM_DIVIDENDS_PER_SHARE = 'CumulativeDividendsPerShare'
    # The ratio of Dividends TTM to Earnings (available to common stockholders) TTM.
    DIVIDEND_PAYOUT_RATIO = 'DividendPayoutRatio'
    # The ratio of Long-term debt to common shareholders' equity (Shareholders equity minus
    # Preferred equity).
    LONG_TERM_DEBT_TO_EQUITY_RATIO = 'LongTermDebtToEquityRatio'
    # The ratio of common shareholders' equity (Shareholders equity minus Preferred equity) to
    # Assets.
    EQUITY_TO_ASSETS_RATIO = 'EquityToAssetsRatio'
    # The ratio of Earnings (available for common stockholders) TTM to Revenue TTM.
    NET_MARGIN = 'NetMargin'
    # The ratio of Revenue TTM to TTM average Assets.
    ASSET_TURNOVER = 'AssetTurnover'
    # Cash from operating activities minus the Capital Expenditures for a quarter.
    FREE_CASH_FLOW_PER_SHARE = 'FreeCashFlowPerShare'
    # The ratio of Current assets to Current liabilities.
    CURRENT_RATIO = 'CurrentRatio'
    
    @staticmethod
    def columns():
        return [getattr(StockPupColumns, col) for col in dir(StockPupColumns) if col[0] != '_' and col != 'columns']

class QuarterlyColumns:
    TICKER_SYMBOL = 'TickerSymbol'
    QUARTER = 'Quarter'
    YEAR = 'Year'
    PRICE_AVG = 'PriceAvg'
    PRICE_HI = 'PriceHigh'
    PRICE_LO = 'PriceLow'
    PRICE_AT_END_OF_QUARTER = 'PriceEoQ'
    AVG_RECOMMENDATIONS = 'AvgRecommendations'
    AVG_RECOMMENDATION_SCORE = 'AvgRecommendationScore'
    SPLIT = 'Split'
    EBIT = 'Ebit'
    PROFIT = 'GrossProfit'
    REVENUE = 'TotalRevenue'
    RND = 'ResearchDevelopment'
    OPERATING_EXPENSES = 'TotalOperatingExpenses'
    INCOME_PRETAX = 'IncomeBeforeTax'
    INCOME_TAX = 'IncomeTaxExpense'
    OPERATING_INCOME = 'OperatingIncome'
    NET_INCOME = 'NetIncome'
    DIVIDENDS = 'DividendsPaid'
    STOCK_REPURCHASED = 'RepurchaseOfStock'
    STOCK_ISSUED = 'IssuanceOfStock'
    DEPRECIATION = 'Depreciation'
    NET_BORROWINGS = 'NetBorrowings'
    INVESTMENTS = 'Investments'
    CASH = 'Cash'
    COMMON_STOCK = 'CommonStock'
    ASSETS = 'TotalAssets'
    LIABILITIES = 'TotalLiab'
    DEBT_LONG = 'LongTermDebt'
    DEBT_SHORT = 'ShortLongTermDebt'
    DATE = 'Date'
    VOLUME = 'Volume'
    EARNINGS = 'Earnings'
    STOCKHOLDER_EQUITY = 'TotalStockholderEquity'
    VOLATILITY = 'Volatility'
    SECTOR = 'Sector'
    INDUSTRY = 'Industry'
    MARKET_CAP = 'MarketCap'
    AGE_OF_DATA = 'AgeOfData'
    WORKING_CAPITAL_RATIO = 'AssetsToLiabilitiesRatio'
    AVG_PE_RATIO = 'AvgPriceToEarningsRatio'
    DEBT_EQUITY_RATIO = 'DebtToEquityRatio'
    ROE = 'ReturnOnEquity'
    PRICE_BOOK_RATIO = 'PriceToBookRatio'
    FCF = 'FreeCashFlow'
    PROFIT_MARGIN = 'ProfitMargin'
    
    @staticmethod
    def columns():
        return [getattr(QuarterlyColumns, col) for col in dir(QuarterlyColumns) if col[0] != '_' and col != 'columns']


QuarterlyIndex = namedtuple('QuarterlyIndex', (QuarterlyColumns.TICKER_SYMBOL, 
                                               QuarterlyColumns.QUARTER, 
                                               QuarterlyColumns.YEAR))
TICKER_SYMBOL, QUARTER, YEAR = 0, 1, 2

PRICE_ONLY_DELTA_COLUMNS = [
    QuarterlyColumns.PRICE_AVG,
    QuarterlyColumns.VOLATILITY
]

DELTA_COLUMNS = [
    QuarterlyColumns.PRICE_AVG,
    QuarterlyColumns.CASH,
    QuarterlyColumns.EARNINGS,
    QuarterlyColumns.AVG_PE_RATIO,
    QuarterlyColumns.DEBT_EQUITY_RATIO,
    QuarterlyColumns.ROE,
    QuarterlyColumns.WORKING_CAPITAL_RATIO,
    QuarterlyColumns.PRICE_BOOK_RATIO,
    QuarterlyColumns.PROFIT_MARGIN,
    QuarterlyColumns.OPERATING_INCOME
]

CATEGORICAL_COLUMNS = [
    QuarterlyColumns.QUARTER,
    QuarterlyColumns.SECTOR,
    QuarterlyColumns.INDUSTRY
]

VS_MARKET_INDICES_COLUMNS = [
    f'{DELTA_PREFIX}{QuarterlyColumns.PRICE_AVG}',
    QuarterlyColumns.VOLATILITY,
]

MARKET_INDICES = ['^DJI']  #, 'VTSAX', '^IXIC', '^GSPC', '^RUT', '^NYA']


FORMULAE = {
    QuarterlyColumns.VOLATILITY: lambda row: (
        row[QuarterlyColumns.PRICE_HI] - row[QuarterlyColumns.PRICE_LO]) / row[QuarterlyColumns.PRICE_AVG],

    QuarterlyColumns.WORKING_CAPITAL_RATIO: lambda row: (
        row[QuarterlyColumns.ASSETS] / row[QuarterlyColumns.LIABILITIES]),

    QuarterlyColumns.AGE_OF_DATA: lambda row: (
        datetime.now().date() - datetime.strptime(row[QuarterlyColumns.DATE], '%Y-%m-%d').date()).days/90,

    QuarterlyColumns.AVG_PE_RATIO: lambda row: (
        row[QuarterlyColumns.PRICE_AVG] / row[QuarterlyColumns.EARNINGS]),

    QuarterlyColumns.DEBT_EQUITY_RATIO: lambda row: (
        row[QuarterlyColumns.DEBT_LONG] + row[QuarterlyColumns.DEBT_SHORT]) / row[QuarterlyColumns.STOCKHOLDER_EQUITY],

    QuarterlyColumns.ROE: lambda row: (
        row[QuarterlyColumns.EARNINGS] - row[QuarterlyColumns.DIVIDENDS]) / row[QuarterlyColumns.STOCKHOLDER_EQUITY],

    QuarterlyColumns.PRICE_BOOK_RATIO: lambda row: (
        row[QuarterlyColumns.ASSETS] - row[QuarterlyColumns.LIABILITIES]) / row[QuarterlyColumns.MARKET_CAP],

    QuarterlyColumns.PROFIT_MARGIN: lambda row: (
        row[QuarterlyColumns.NET_INCOME] / row[QuarterlyColumns.REVENUE])
}

TARGET_COL = f'{DELTA_PREFIX}{QuarterlyColumns.PRICE_AVG}{VS_MKT_IDX}^DJI'

FEATURE_COLS = [
#     QuarterlyColumns.QUARTER,
    QuarterlyColumns.SECTOR,
    QuarterlyColumns.AGE_OF_DATA,
    QuarterlyColumns.VOLATILITY,
    QuarterlyColumns.AVG_RECOMMENDATION_SCORE,
    QuarterlyColumns.AVG_PE_RATIO,
    QuarterlyColumns.DEBT_EQUITY_RATIO,
    QuarterlyColumns.ROE,
    QuarterlyColumns.WORKING_CAPITAL_RATIO,
    QuarterlyColumns.PRICE_BOOK_RATIO,
    QuarterlyColumns.OPERATING_INCOME,
    QuarterlyColumns.PROFIT_MARGIN
] + [
    f'{DELTA_PREFIX}{col}' for col in DELTA_COLUMNS
] + [
    f'{QuarterlyColumns.VOLATILITY}{VS_MKT_IDX}{mkt_idx}' for mkt_idx in MARKET_INDICES
]

In [133]:
db_conn = sqlite3.connect(DB_PATH)

quarterly_df = pd.read_sql_query(f'SELECT * FROM {TABLE_NAME}', db_conn)
db_conn.close()

# This needs to be above the filters below otherwise we'll drop quarterly data! (indices have no revenue)
market_index_df = quarterly_df[quarterly_df[QuarterlyColumns.TICKER_SYMBOL].isin(MARKET_INDICES)]

market_index_df.dropna(subset=[QuarterlyColumns.DATE,
                               QuarterlyColumns.PRICE_AVG,
                               QuarterlyColumns.PRICE_HI,
                               QuarterlyColumns.PRICE_LO,
                               ])

quarterly_df.dropna(subset=[QuarterlyColumns.DATE,
                            QuarterlyColumns.REVENUE,
                            QuarterlyColumns.PRICE_AVG,
                            QuarterlyColumns.PRICE_HI,
                            QuarterlyColumns.PRICE_LO,
                            QuarterlyColumns.EARNINGS],
                    inplace=True)
quarterly_df = quarterly_df[((quarterly_df[QuarterlyColumns.REVENUE] != 0) &
                             (quarterly_df[QuarterlyColumns.EARNINGS] != 0) &
                             (~quarterly_df[QuarterlyColumns.TICKER_SYMBOL].isin(MARKET_INDICES)))]

quarterly_df.set_index([QuarterlyColumns.TICKER_SYMBOL,
                        QuarterlyColumns.QUARTER,
                        QuarterlyColumns.YEAR],
                       inplace=True)
quarterly_df.sort_index(inplace=True)



In [134]:
db_conn = sqlite3.connect(DB_PATH)

stockpup_df = pd.read_sql_query(f'SELECT * FROM {STOCKPUP_TABLE_NAME}', db_conn)
db_conn.close()


def _date_to_index(dt: datetime.date):
    q = MONTH_TO_QUARTER[dt.month]
    year = dt.year - 1 if dt.month == 1 else dt.year
    return QuarterlyIndex('', q, year)


def process_stockpup_df(df):
    df.dropna(subset=[StockPupColumns.SHARES, 
                      StockPupColumns.SHARES_SPLIT_ADJUSTED,
                      StockPupColumns.FREE_CASH_FLOW_PER_SHARE,
                      StockPupColumns.EARNINGS,
                      StockPupColumns.SHAREHOLDER_EQUITY,
                      StockPupColumns.LIABILITIES,
                      StockPupColumns.PRICE],
              inplace=True)
    df = df[((df[StockPupColumns.REVENUE] != 0) &
             (df[StockPupColumns.EARNINGS] != 0))]
    
    df[StockPupColumns.QUARTER_END] = pd.to_datetime(df[StockPupColumns.QUARTER_END])

    df[QuarterlyColumns.QUARTER] = df[StockPupColumns.QUARTER_END].apply(
        lambda r: _date_to_index(r)[QUARTER])
    df[QuarterlyColumns.YEAR] = df[StockPupColumns.QUARTER_END].apply(lambda r: _date_to_index(r)[YEAR])
    df[QuarterlyColumns.DIVIDENDS] = df[StockPupColumns.DIVIDEND_PER_SHARE] * df[
        StockPupColumns.SHARES]
    df[QuarterlyColumns.DATE] = df[StockPupColumns.QUARTER_END].apply(lambda r: str(r.date()))
    df[QuarterlyColumns.OPERATING_INCOME] = df[StockPupColumns.FREE_CASH_FLOW_PER_SHARE] * df[
        StockPupColumns.SHARES]
    df[QuarterlyColumns.MARKET_CAP] = df[StockPupColumns.SHARES_SPLIT_ADJUSTED] * df[StockPupColumns.PRICE]
    df[QuarterlyColumns.DEBT_SHORT] = 0  # I think short long term debt is figured into long term debt
#     df[QuarterlyColumns.EBIT] = ???  # TODO: Compute EBIT?

    df.rename(columns={
        StockPupColumns.ASSETS: QuarterlyColumns.ASSETS,
        StockPupColumns.REVENUE: QuarterlyColumns.REVENUE,
        StockPupColumns.EARNINGS: QuarterlyColumns.EARNINGS,
        StockPupColumns.LIABILITIES: QuarterlyColumns.LIABILITIES,
        StockPupColumns.LONG_TERM_DEBT: QuarterlyColumns.DEBT_LONG,
        StockPupColumns.SHAREHOLDER_EQUITY: QuarterlyColumns.STOCKHOLDER_EQUITY,
        StockPupColumns.CASH_AT_END_OF_PERIOD: QuarterlyColumns.CASH,
        StockPupColumns.PRICE: QuarterlyColumns.PRICE_AVG,
        StockPupColumns.PRICE_LOW: QuarterlyColumns.PRICE_LO,
        StockPupColumns.PRICE_HIGH: QuarterlyColumns.PRICE_HI,
        StockPupColumns.SPLIT_FACTOR: QuarterlyColumns.SPLIT,
        StockPupColumns.SHARES_SPLIT_ADJUSTED: QuarterlyColumns.COMMON_STOCK
    }, inplace=True)
    
    df[QuarterlyColumns.DEBT_LONG] = df[QuarterlyColumns.DEBT_LONG].apply(lambda row: int(row))

    # Filter only to columns in QuarterlyColumns
    df = df[[col for col in df.columns if col in QuarterlyColumns.columns()]]
    
    df.set_index([QuarterlyColumns.TICKER_SYMBOL,
                  QuarterlyColumns.QUARTER,
                  QuarterlyColumns.YEAR],
                 inplace=True)
    df.sort_index(inplace=True)
    
    # Filter out all data in stockpup_df that exists in quarterly_df (by index)
    df = df[~df.index.isin(quarterly_df.index)]
    
    return df

stockpup_df = process_stockpup_df(stockpup_df)

quarterly_df = pd.concat([quarterly_df, stockpup_df])
quarterly_df.sort_index(inplace=True)
quarterly_df

/Users/ezras/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ezras/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ezras/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

PriceAvg  PriceHigh  PriceLow  PriceEoQ  \
TickerSymbol Quarter Year                                              
A            1       2000  115.000000     159.00     71.00       NaN   
                     2002   31.410000      38.00     24.83       NaN   
                     2003   14.060000      16.82     11.30       NaN   
                     2004   32.270000      37.62     26.91       NaN   
                     2005   22.550000      24.99     20.11       NaN   
...                               ...        ...       ...       ...   
ZTS          4       2015   43.810000      48.65     38.98       NaN   
                     2016   50.500000      54.15     46.86       NaN   
                     2017   68.310000      73.58     63.03       NaN   
                     2018   87.730000      96.57     78.90       NaN   
                     2019  123.596032     132.84    115.89    132.84   

                                                          AvgRecommendations  \
TickerSymbol Quarter Year                                                      
A            1       2000                                                NaN   
                     2002                                                NaN   
                     2003                                                NaN   
                     2004                                                NaN   
                     2005                                                NaN   
...                                                                      ...   
ZTS          4       2015                                                NaN   
                     2016                                                NaN   
                     2017                                                NaN   
                     2018                                                NaN   
                     2019  {"BMOCapital": 0, "BankofAmerica": 0, "Barclay...   

                          Split         Ebit   GrossProfit  TotalRevenue  \
TickerSymbol Quarter Year                                                  
A            1       2000     1          NaN           NaN  2.485000e+09   
                     2002     1          NaN           NaN  1.457000e+09   
                     2003     1          NaN           NaN  1.467000e+09   
                     2004     1          NaN           NaN  1.831000e+09   
                     2005     1          NaN           NaN  1.688000e+09   
...                         ...          ...           ...           ...   
ZTS          4       2015     1          NaN           NaN  1.274000e+09   
                     2016     1          NaN           NaN  1.277000e+09   
                     2017     1          NaN           NaN  1.460000e+09   
                     2018     1          NaN           NaN  1.564000e+09   
                     2019  None  599000000.0  1.169000e+09  1.674000e+09   

                           ResearchDevelopment  TotalOperatingExpenses  \
TickerSymbol Quarter Year                                                
A            1       2000                  NaN                     NaN   
                     2002                  NaN                     NaN   
                     2003                  NaN                     NaN   
                     2004                  NaN                     NaN   
                     2005                  NaN                     NaN   
...                                        ...                     ...   
ZTS          4       2015                  NaN                     NaN   
                     2016                  NaN                     NaN   
                     2017                  NaN                     NaN   
                     2018                  NaN                     NaN   
                     2019          132000000.0            1.075000e+09   

                           IncomeBeforeTax  IncomeTaxExpense  OperatingIncome  \
TickerSymbol Quart

In [135]:
stock_info_df = pd.read_csv(INFO_CSV_PATH)[['tickerSymbol', 'sector', 'industry']]
stock_info_df.rename(columns={
    'tickerSymbol': QuarterlyColumns.TICKER_SYMBOL,
    'sector': QuarterlyColumns.SECTOR,
    'industry': QuarterlyColumns.INDUSTRY
}, inplace=True)
stock_info_df.set_index([QuarterlyColumns.TICKER_SYMBOL], inplace=True)

quarterly_df = quarterly_df.join(stock_info_df, on=[QuarterlyColumns.TICKER_SYMBOL])

quarterly_df[QuarterlyColumns.SECTOR].fillna(MISSING_SECTOR, inplace=True)
quarterly_df[QuarterlyColumns.INDUSTRY].fillna(MISSING_INDUSTRY, inplace=True)
quarterly_df[QuarterlyColumns.DEBT_SHORT].fillna(0, inplace=True)

In [136]:
for col_name, fn in FORMULAE.items():
        quarterly_df[col_name] = quarterly_df.apply(fn, axis=1)

market_index_df[QuarterlyColumns.VOLATILITY] = market_index_df.apply(FORMULAE[QuarterlyColumns.VOLATILITY], axis=1)
market_index_df.set_index([QuarterlyColumns.TICKER_SYMBOL,
                           QuarterlyColumns.QUARTER,
                           QuarterlyColumns.YEAR],
                          inplace=True)
market_index_df.sort_index(inplace=True)

quarterly_df

PriceAvg  PriceHigh  PriceLow  PriceEoQ  \
TickerSymbol Quarter Year                                              
A            1       2000  115.000000     159.00     71.00       NaN   
                     2002   31.410000      38.00     24.83       NaN   
                     2003   14.060000      16.82     11.30       NaN   
                     2004   32.270000      37.62     26.91       NaN   
                     2005   22.550000      24.99     20.11       NaN   
...                               ...        ...       ...       ...   
ZTS          4       2015   43.810000      48.65     38.98       NaN   
                     2016   50.500000      54.15     46.86       NaN   
                     2017   68.310000      73.58     63.03       NaN   
                     2018   87.730000      96.57     78.90       NaN   
                     2019  123.596032     132.84    115.89    132.84   

                                                          AvgRecommendations  \
TickerSymbol Quarter Year                                                      
A            1       2000                                                NaN   
                     2002                                                NaN   
                     2003                                                NaN   
                     2004                                                NaN   
                     2005                                                NaN   
...                                                                      ...   
ZTS          4       2015                                                NaN   
                     2016                                                NaN   
                     2017                                                NaN   
                     2018                                                NaN   
                     2019  {"BMOCapital": 0, "BankofAmerica": 0, "Barclay...   

                          Split         Ebit   GrossProfit  TotalRevenue  \
TickerSymbol Quarter Year                                                  
A            1       2000     1          NaN           NaN  2.485000e+09   
                     2002     1          NaN           NaN  1.457000e+09   
                     2003     1          NaN           NaN  1.467000e+09   
                     2004     1          NaN           NaN  1.831000e+09   
                     2005     1          NaN           NaN  1.688000e+09   
...                         ...          ...           ...           ...   
ZTS          4       2015     1          NaN           NaN  1.274000e+09   
                     2016     1          NaN           NaN  1.277000e+09   
                     2017     1          NaN           NaN  1.460000e+09   
                     2018     1          NaN           NaN  1.564000e+09   
                     2019  None  599000000.0  1.169000e+09  1.674000e+09   

                           ResearchDevelopment  TotalOperatingExpenses  \
TickerSymbol Quarter Year                                                
A            1       2000                  NaN                     NaN   
                     2002                  NaN                     NaN   
                     2003                  NaN                     NaN   
                     2004                  NaN                     NaN   
                     2005                  NaN                     NaN   
...                                        ...                     ...   
ZTS          4       2015                  NaN                     NaN   
                     2016                  NaN                     NaN   
                     2017                  NaN                     NaN   
                     2018                  NaN                     NaN   
                     2019          132000000.0            1.075000e+09   

                           IncomeBeforeTax  IncomeTaxExpense  OperatingIncome  \
TickerSymbol Quart

In [ ]:
def get_prev_quarterly_index(index: QuarterlyIndex):
    if index[QUARTER] == 1:
        return QuarterlyIndex(index[TICKER_SYMBOL], 4, index[YEAR] - 1)

    return QuarterlyIndex(index[TICKER_SYMBOL], index[QUARTER] - 1, index[YEAR])


def add_delta_columns(row: pd.Series, df: pd.DataFrame, columns: list):    
    try:
        prev_quarter_row = df.loc[get_prev_quarterly_index(row.name)]  # row.name returns the multiIndex tuple
    except:
#         print(f'Unable to find prev quarter info for {row.name}')
        prev_quarter_row = pd.DataFrame()
    
    if not prev_quarter_row.empty:
        new_cols = []

        for col in columns:
            # converting to float to get rid of index terms
            new_cols.append(float((row[col] - prev_quarter_row[col]) / prev_quarter_row[col]))

        return pd.Series(new_cols)

    return pd.Series([None]*len(columns))

def add_delta_columns_quarterly_df(row: pd.Series):
    return add_delta_columns(row, quarterly_df, DELTA_COLUMNS)

def add_delta_columns_market_index_df(row: pd.Series):
    return add_delta_columns(row, market_index_df, PRICE_ONLY_DELTA_COLUMNS)


delta_col_names = [f'{DELTA_PREFIX}{col}' for col in DELTA_COLUMNS]
quarterly_df[delta_col_names] = quarterly_df.apply(add_delta_columns_quarterly_df, axis=1)
quarterly_df.dropna(subset=delta_col_names, inplace=True)

delta_col_mkt_index_names = [f'{DELTA_PREFIX}{col}' for col in PRICE_ONLY_DELTA_COLUMNS]
market_index_df[delta_col_mkt_index_names] = market_index_df.apply(add_delta_columns_market_index_df, axis=1)
market_index_df.dropna(subset=delta_col_mkt_index_names, inplace=True)

In [111]:
def compare_to_market_index(row: pd.Series, market_indices=None):
    market_indices = MARKET_INDICES if not market_indices else market_indices
    
    new_cols = []
    for col in VS_MARKET_INDICES_COLUMNS:
        for mkt_idx in market_indices:
            try:
                mkt_idx_row = market_index_df.loc[mkt_idx, row.name[QUARTER], row.name[YEAR]]
            except:
#                 print(f'Unable to find {mkt_idx} Q{row.name[QUARTER]} {row.name[YEAR]}')
                mkt_idx_row = pd.DataFrame()
            
            if not mkt_idx_row.empty:
                # converting to float to drop index terms
                new_cols.append(float(row[f'{col}'] / mkt_idx_row[f'{col}']))
            else:
                new_cols.append(None)

    return pd.Series(new_cols)

vs_market_indices_col_names = [f'{col}{VS_MKT_IDX}{mkt_idx}' 
                               for col in VS_MARKET_INDICES_COLUMNS for mkt_idx in MARKET_INDICES]
quarterly_df[vs_market_indices_col_names] = quarterly_df.apply(compare_to_market_index, axis=1)
quarterly_df.dropna(subset=vs_market_indices_col_names, inplace=True)

In [112]:
quarterly_df.shape

(59205, 55)

In [117]:
def get_avg_recommendation_score(row: pd.Series):
    if row[QuarterlyColumns.AVG_RECOMMENDATIONS] is None or str(row[QuarterlyColumns.AVG_RECOMMENDATIONS]) == 'nan':
        return pd.Series([None])
    
    avg_recommendation = np.mean([float(v) for v in json.loads(row[QuarterlyColumns.AVG_RECOMMENDATIONS]).values()])
    return pd.Series([avg_recommendation])


quarterly_df[QuarterlyColumns.AVG_RECOMMENDATION_SCORE] = quarterly_df.apply(get_avg_recommendation_score, axis=1)

In [387]:
recommendations = [json.loads(contents) if contents else {} 
                   for contents in quarterly_df[QuarterlyColumns.AVG_RECOMMENDATIONS]]
recommendations = [
    {f'{AVG_REC_SCORE_PREFIX}{firm}': value if value is not None else 0 for firm, value in recommendation.items()} 
    for recommendation in recommendations]
recommendations_df = pd.DataFrame(recommendations).fillna(0)


quarterly_df = pd.merge(quarterly_df, recommendations_df, left_index=True, right_index=True)

In [122]:
feature_df = quarterly_df[FEATURE_COLS]
feature_df

Sector  AgeOfData  Volatility  \
TickerSymbol Quarter Year                                      
A            1       2000  Healthcare  81.866667    0.765217   
                     2002  Healthcare  73.755556    0.419293   
                     2003  Healthcare  69.700000    0.392603   
                     2004  Healthcare  65.633333    0.331887   
                     2005  Healthcare  61.577778    0.216408   
...                               ...        ...         ...   
ZTS          4       2015  Healthcare  18.277778    0.220726   
                     2016  Healthcare  14.211111    0.144356   
                     2017  Healthcare  10.155556    0.154443   
                     2018  Healthcare   6.100000    0.201413   
                     2019  Healthcare   2.044444    0.137140   

                           AvgRecommendationScore  AvgPriceToEarningsRatio  \
TickerSymbol Quarter Year                                                    
A            1       2000                     NaN             6.927711e-07   
                     2002                     NaN            -1.241502e-07   
                     2003                     NaN            -9.630137e-08   
                     2004                     NaN             3.102885e-07   
                     2005                     NaN             2.373684e-07   
...                                           ...                      ...   
ZTS          4       2015                     NaN             1.991364e-06   
                     2016                     NaN             3.279221e-07   
                     2017                     NaN             8.433333e-07   
                     2018                     NaN             2.542899e-07   
                     2019                0.416667             2.854412e-07   

                           DebtToEquityRatio  ReturnOnEquity  \
TickerSymbol Quarter Year                                      
A            1       2000           0.000000        0.035760   
                     2002           0.224259       -0.049337   
                     2003           0.273744       -0.034754   
                     2004           0.360841        0.032633   
                     2005           0.297234        0.024554   
...                                      ...             ...   
ZTS          4       2015           4.178839       -0.021296   
                     2016           3.004707        0.037396   
                     2017           2.798305       -0.017293   
                     2018           2.948741        0.094351   
                     2019           2.382939        0.189069   

                           AssetsToLiabilitiesRatio  PriceToBookRatio  \
TickerSymbol Quarter Year                                               
A            1       2000                  2.732736          0.089250   
                     2002                  2.648875          0.351666   
                     2003                  2.229801          0.633966   
                     2004                  1.913704          0.205131   
                     2005                  2.138276          0.349185   
...                                             ...               ...   
ZTS          4       2015                  1.159924          0.050091   
                     2016                  1.243740          0.060336   
                     2017                  1.262647          0.053880   
                     2018                  1.254306          0.052021   
                     2019                  1.306439          0.041866   

                           OperatingIncome  ProfitMargin  \
TickerSymbol Quarter Year                                  
A            1       2000    -1.311589e+08           NaN   
                     2002    -2.042689e+08           NaN   
                     2003    -2.215130e+08           NaN   
                     2004     1.444353e+08           NaN   
                     2005     2.014563e+

73

In [38]:
pd.set_option('display.max_rows', 30)

In [130]:
quarterly_df['Delta_ReturnOnEquity'][0]

0.2245906114434751

(61750,)